Формула для характеристической функции модели Коу:
$$
\phi(\xi)=e^{-t\psi(\xi)},
$$
где
$$
\psi(\xi)=\frac{\sigma^2}{2} \xi^2-i \mu \xi+\frac{i c_{+} \xi}{\lambda_{+}+i \xi}+\frac{i c_{-} \xi}{\lambda_{-}+i \xi}
$$

$$
\mu = r - \frac{1}{2} \sigma^2 - \frac{p \cdot \lambda}{\Lambda_+ - 1} + \frac{(1 - p) \cdot \lambda}{\Lambda_- + 1}
$$


In [16]:
def compute_mu(r, sigma, intensity, lambda_p, lambda_m, p):
    kappa_plus = (p * intensity) / (lambda_p - 1)
    kappa_minus = -1 * (1 - p) * intensity / (lambda_m + 1)
    kappa = kappa_plus + kappa_minus
    mu = r - 0.5 * sigma ** 2 - kappa
    return mu

In [17]:
########## Вспомогательные функции ##########
def kou_chexp(xi, z, sigma, cp, cm, lm, lp, mu):
    xi = xi + z * 1j
    sig2 = 0.5 * sigma ** 2
    temp1 = sig2 * xi ** 2 + (-mu * xi * 1j)
    temp2 = (cp * xi * 1j) / (xi * 1j + lp)
    temp3 = (cm * xi * 1j) / (xi * 1j + lm)
    return temp1 + temp2 + temp3

def kou_chf(xi, z, t, sigma, cp, cm, lm, lp, mu):
    temp1 = kou_chexp(xi, z, sigma, cp, cm, lm, lp, mu)
    return cmath.exp(-t * temp1)

Для вычисления цены опциона необходимо воспользоваться следующей формулой:
$$
F\left(x_j, T\right)=\frac{1}{2 \pi} \int_{-\infty+i z}^{+\infty+i z} e^{i x_j \xi} e^{ -T \psi(\xi)} \hat{g}(\xi) d \xi,
$$ предварительно сделав замену $\xi=\xi+i z$ и вычислив $\hat{g}(\xi) = \hat{g}(\xi + iz) = \frac{-K}{(\xi_k + iz)(\xi_k + i(1+z))}$


Для IFFT будет использоваться вектор $v_F$, элементами которого являются выражения вида

$$
\phi_{X_T}(\xi+iz)\cdot(-1)^k \cdot \hat{g}(\xi + iz),\quad k = 0,...,M-1
$$

Тогда получаем следующую формулу:
$$
F(x_j, T) = -Ke^{-x_jz}e^{-rT}\cdot(-1)^j\frac{1}{d} iDFT[v_F](j), \quad j = 0,...,M-1
$$


In [22]:
import numpy as np
import cmath
import matplotlib.pyplot as plt
########## Основной алгоритм ##########

# Параметры алгоритма
m = 10
d = 0.01

# Параметры
cp, cm, intensity, lp, lm = 1, 1, 12, 10, -15
sigma, p, r = 0.3, 0.5, 0.05
mu = compute_mu(r, sigma, intensity, -lm, lp, p)
T = 10/252

S = 100  # Начальная цена
K = 80  # Страйк
M = int(2 ** m)

xi = -np.pi / d  # текущее значение xi
dxi = -2 * xi / M  # шаг в пространстве xi
z = -2  # сдвиг для вычисления call-опциона

# Создание массивов для Фурье-преобразования
v_F = np.zeros(M, dtype=complex)
v_x = np.zeros(M, dtype=complex)

# Интеграл обратного преобразования Фурье
sign_out = 1
for k in range(M):
    phi = kou_chf(xi, z, T, sigma, cp, cm, lm, lp, mu)
    F = sign_out * phi
    v_F[k] = F / ((xi + 1j * z) * (xi + 1j * (z + 1)))

    xi += dxi
    sign_out = -sign_out

#IFFT
v_x = np.fft.ifft(v_F)
x_target = np.log(S/K)
j = int(M/2 + x_target/d)
x = -M*d/2 + j*d
price = -v_x[j].real * K * np.exp(-x * z) * np.exp(-r * T) / d
print(price)

20.27148687440624
